In [1]:
import matplotlib.pyplot as plt
from capstone_group4 import load_and_inspect_data, perform_linear_regression_analysis

# --- Main excution ---
if __name__ == '__main__':
    # Chart type
    plt.style.use('seaborn-v0_8-whitegrid')
    
    # Define file path
    file_path = 'C:/Users/frank/OneDrive/Documents/mlb_salaries_and_pitching_stats_2011-2024.csv'
    
    # Step 1-Load data and inspect data
    main_df = load_and_inspect_data(file_path)
    
    # Step 2- if success, run regression model
    if main_df is not None:
        perform_linear_regression_analysis(main_df)
        print("\nAnalysis complete.")

ModuleNotFoundError: No module named 'capstone_group4'